In [ ]:
import os
import numpy as np

from cloe.user_interface.likelihood_ui import LikelihoodUI
from cloe.auxiliary                    import likelihood_yaml_handler as lyh
from cloe.auxiliary.yaml_handler       import yaml_read,yaml_write

from copy import deepcopy

# Options

Here you can decide the extra cosmologies to be added (both for mcmc_scripts and config files) and the combination of observables to be considered (only for mcmc_scripts)

## Paths and settings

**Scripts creation settings**

- main_path: the path to the folder where the config defaults are placed;
- subfolder: subfolder of the above where to create the new config files;
- modelfolder: subfolder of configs/models/ where the new cosmology files will be created;
- mcmc_folder: path where to create the .py scripts to run MCMC.

**Options visualization settings**

- layer_limit: this number sets a limit for the number of subdictionaries to be visualized (>=10 should give the full dictionary);
- show_params: if set to False, the details of the parameters (fiducial, prior, ...) are not visualized, and only a list of the selectable parameters is provided.

In [ ]:
main_path   = '../configs/'
subfolder   = 'extra_config_cases/'
modelfolder = 'extra_cosmologies/'
mcmc_folder = '../mcmc_scripts/'

if not os.path.exists(main_path+subfolder):
    os.makedirs(main_path+subfolder)
    
if not os.path.exists(main_path+'models/'+modelfolder):
    os.makedirs(main_path+'models/'+modelfolder)
    
if not os.path.exists(mcmc_folder):
    os.makedirs(mcmc_folder)

layer_limit = 10
show_params = True

## Extra parameters

Here you can set the details for the additional parameters sampling and which extra cosmologies you want to consider. 

In [ ]:
w0_dict = {'latex': 'w_0',
           'prior': {'max': -0.5, 'min': -3.},
           'proposal': 0.5,
           'ref': {'dist': 'norm', 'loc': -1, 'scale': 0.5}}

wa_dict = {'latex': 'w_a',
           'prior': {'max': 0.5, 'min': -3.},
           'proposal': 0.1,
           'ref': {'dist': 'norm', 'loc': 0, 'scale': 0.1}}

omk_dict = {'latex': '\Omega_k',
            'prior': {'max': 0.1, 'min': -0.1},
            'proposal': 0.05,
            'ref': {'dist': 'norm', 'loc': 0, 'scale': 0.05}}

gamma_dict = {'latex': '\gamma',
              'prior': {'max': 1., 'min': 0.1},
              'proposal': 0.1,
              'ref': {'dist': 'norm', 'loc': 0.55, 'scale': 0.1}}



extra_cosmologies = {'LCDM': None,
                     'LCDM_nonflat': {'omk': omk_dict},
                     'wCDM': {'w': w0_dict},
                     'w0waCDM': {'w': w0_dict,
                                 'wa': wa_dict},
                     'w0waCDM_nonflat': {'w': w0_dict,
                                         'wa': wa_dict,
                                         'omk': omk_dict},
                     'gamma_w0waCDM': {'w': w0_dict,
                                       'wa': wa_dict,
                                       'gamma_MG': gamma_dict},
                     'gamma_w0waCDM_nonflat': {'w': w0_dict,
                                               'wa': wa_dict,
                                               'omk': omk_dict,
                                               'gamma_MG': gamma_dict}}

## Observables

Using the Santiago Matrix layout, set the observable combinations to use to produce the MCMC scripts.

**WARNING!** this is used **ONLY** for the MCMC scripts, the config files are automatically created only for the full combination (GCsp+3x2pt) as they were until now.

In [ ]:
obs_combination = {'3x2pt': {'WL': {'WL': True,
                                    'GCphot': True,
                                    'GCspectro': False},
                             'GCphot': {'GCphot': True,
                                        'GCspectro': False},
                             'GCspectro': {'GCspectro': False}},
                   '3x2pt+GCsp': {'WL': {'WL': True,
                                         'GCphot': True,
                                         'GCspectro': False},
                                  'GCphot': {'GCphot': True,
                                             'GCspectro': False},
                                  'GCspectro': {'GCspectro': True}}}

# Reading the default config file

This reads the default config, model and cosmology files. 

If these are kept up to date, this notebook should be able to catch up any additional flag or option.

In [ ]:
ui = LikelihoodUI()

default = {'config': yaml_read(main_path+'config_default.yaml')}
default['model'] = yaml_read(main_path+default['config']['Cobaya']['params'])
default['cosmo'] = yaml_read(main_path+'models/'+default['model']['user_models']['cosmology'])

# Creating config files for interesting cosmologies

Creates the new config, model and cosmology files for the models described above.

*Our yaml handler automatically prints a warning when overwriting a file, so such a warning will always appear here* 

In [ ]:
for newset in extra_cosmologies:    
    newdict = deepcopy(default)

    newdict['config'].pop('define')
    
    newdict['config']['Cobaya']['params'] = './'+subfolder+'model_'+newset+'.yaml'
    newdict['model']['user_models']['cosmology'] = modelfolder+'cosmology_'+newset+'.yaml'
    if 'gamma' in newset:
        newdict['config']['Cobaya']['likelihood']['Euclid']['use_gamma_MG'] = True
    else:
        newdict['config']['Cobaya']['likelihood']['Euclid']['use_gamma_MG'] = False
        
    newdict['config']['Cobaya']['output'] = 'chains/my_euclid_experiment_'+newset
        
    if extra_cosmologies[newset] != None:
        newdict['cosmo'].update({par: extra_cosmologies[newset][par] 
                                 for par in extra_cosmologies[newset]})
        
    for name in newdict:
        if name == 'cosmo':
            filepath = main_path+'models/'+modelfolder+'cosmology_'+newset+'.yaml'
        else:
            filepath = main_path+subfolder+name+'_'+newset+'.yaml'
        yaml_write(filepath,newdict[name],overwrite=True)

# Creating mcmc scripts for different cosmologies and observables

This part of the notebook reads in **ALL** the default information, includes the new cosmologies and observable combination and prints the MCMC scripts for what is selected.

## Functions

These are used to print the dictionary to file in a decently humanly readable form

In [ ]:
def pretty_dict(f,dic,layer=0):
    for key, value in dic.items():
        if isinstance(value,dict):
            if isinstance(key,str):
                f.write(" "*(4*layer+8)+"'{}':\n".format(key))
            else:
                f.write(" "*(4*layer+8)+"{}:\n".format(key))
            f.write(" "*(4*layer+8)+"{\n")
            pretty_dict(f,value,layer=layer+1)
            f.write(" "*(4*layer+8)+"},\n")
        else:
            if isinstance(value,str) and value != 'EuclidLikelihood':
                f.write(" "*(4*layer+8)+"'{}': '{}',\n".format(key,value))
            else:
                f.write(" "*(4*layer+8)+"'{}': {},\n".format(key,value))

def print_script(filename,dictionary):

    with open(filename, 'w') as f:
        f.write("import sys\n")
        f.write("import os\n")
        f.write("from cloe.auxiliary.likelihood_yaml_handler \\\n")
        f.write("\timport write_params_yaml_from_info_dict\n")
        f.write("\n")
        f.write("script_path = os.path.realpath(os.getcwd())\n")
        f.write("if script_path.endswith('mcmc_scripts'):\n")
        f.write("    sys.path.append(os.path.realpath(os.path.join(script_path, os.pardir)))\n")
        f.write("else:\n")
        f.write("    sys.path.append(script_path)\n")
        f.write("\n")
        f.write("from cobaya.run import run\n")
        f.write("from cloe.cobaya_interface import EuclidLikelihood\n")
        f.write("\n")
        f.write("info = {\n")
        pretty_dict(f,dictionary)
        f.write("}\n")
        f.write("\n")
        f.write("write_params_yaml_from_info_dict(info)\n")
        f.write("\n")
        f.write("updated_info, sampler = run(info)\n")

In [ ]:
default_full_dict = deepcopy(ui._config)
likelihood_euclid_dict = default_full_dict['Cobaya']['likelihood']['Euclid']

ui._check_and_update_likelihood_fields(likelihood_euclid_dict)
ui._check_and_update_params_field(default_full_dict['Cobaya'])
lyh.update_cobaya_dict_with_halofit_version(default_full_dict['Cobaya'])

fixed_dict = deepcopy(default_full_dict)

for comb in obs_combination:
    for model in extra_cosmologies:

        filename = mcmc_folder+'runmcmc_'+comb+'_'+model+'.py'
        final_dict = deepcopy(fixed_dict)
        if extra_cosmologies[model] != None:
            for par in extra_cosmologies[model]:
                final_dict['Cobaya']['params'][par] = extra_cosmologies[model][par]
            
        final_dict['Cobaya']['likelihood']['Euclid']['observables_selection'] = obs_combination[comb]

        final_dict['Cobaya']['output'] = './chains/chain_'+comb+'_'+model
        final_dict['Cobaya']['likelihood']['Euclid']['external'] = 'EuclidLikelihood'
        
        print_script(filename,final_dict['Cobaya'])

# List all available options

This part of the notebook visualizes on screen all the possible settings that can be selected in CLOE+Cobaya.

Options on the level of detail reached by this script are available in the settings above.

In [ ]:
def print_keys(dic,layer=0):
    for key, value in dic.items():
        if isinstance(value,dict) and layer<=layer_limit:
            print('')
            print(' '*(4*layer)+'{}:'.format(key))
            if key == 'params' and not show_params:
                print(' '*(4*layer)+'Only printing list of available params, no detail')
                for par in value:
                    print(' '*(4*layer)+'{}'.format(par))
            else:
                print_keys(value,layer=layer+1)
        elif layer>layer_limit:
            print(' '*(4*layer)+'{}: ...'.format(key))
        else:
            print(' '*(4*layer)+'{}: {}'.format(key,value))

In [ ]:
default_full_dict = deepcopy(ui._config)
likelihood_euclid_dict = default_full_dict['Cobaya']['likelihood']['Euclid']

ui._check_and_update_likelihood_fields(likelihood_euclid_dict)
ui._check_and_update_params_field(default_full_dict['Cobaya'])
lyh.update_cobaya_dict_with_halofit_version(default_full_dict['Cobaya'])

print_keys(default_full_dict)